In [1]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pandas as pd

In [6]:
## API KEY: LORENZO

# open the text file containing the API key
with open("../authentication/YouTube_Data_API_Key.txt", "r") as file:
    api_key = file.read().strip() 

MAX_RESULTS = 30

In [18]:
def search_videos(query, max_results=MAX_RESULTS, published_after=None, published_before=None):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # convert datetime objects to ISO 8601 string format
    published_after_string = published_after.strftime('%Y-%m-%dT%H:%M:%SZ') if published_after else None
    published_before_string = published_before.strftime('%Y-%m-%dT%H:%M:%SZ') if published_before else None

    search_request_medium = youtube.search().list(
        q=query, 
        part="snippet", 
        type="video", 
        maxResults=max_results, 
        order="viewCount",
        videoDuration="medium", 
        publishedAfter=published_after_string, 
        publishedBefore=published_before_string)
    
    search_request_long = youtube.search().list(
        q=query, 
        part="snippet", 
        type="video", 
        maxResults=max_results, 
        order="viewCount",
        videoDuration="long", 
        publishedAfter=published_after_string, 
        publishedBefore=published_before_string)
    
    search_response_medium = search_request_medium.execute()
    search_response_long = search_request_long.execute()

    video_ids_medium = [item['id']['videoId'] for item in search_response_medium['items']]
    video_ids_long = [item['id']['videoId'] for item in search_response_long['items']]

    video_request_medium = youtube.videos().list(part="snippet,statistics", id=",".join(video_ids_medium))
    video_request_long = youtube.videos().list(part="snippet,statistics", id=",".join(video_ids_long))

    video_response_medium = video_request_medium.execute()
    video_response_long = video_request_long.execute()

    videos_data = []
    for item in video_response_medium['items']:
        videos_data.append({
            'title': item['snippet']['title'],
            'publish_date': item['snippet']['publishedAt'],
            'description': item['snippet']['description'],
            'video_id': item['id'],
            'view_count': int(item['statistics']['viewCount']) if 'viewCount' in item['statistics'] else 0
        })
    
    for item in video_response_long['items']:
        videos_data.append({
            'title': item['snippet']['title'],
            'publish_date': item['snippet']['publishedAt'],
            'description': item['snippet']['description'],
            'video_id': item['id'],
            'view_count': int(item['statistics']['viewCount']) if 'viewCount' in item['statistics'] else 0
        })

    return pd.DataFrame(videos_data)

In [20]:
start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 12, 31)
videos_df = search_videos("artificial intelligence", MAX_RESULTS, start_date, end_date)

In [29]:
videos_df.head(10)

KeyError: 'new_column'

In [27]:
def get_video_comments(api_key, video_id, max_results=20):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Call the API to retrieve comments
    response = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=max_results
    ).execute()

    comments = []
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
        comments.append(comment)

    return comments

In [28]:
comments = get_video_comments(api_key, "Bg_tJvCA8zw")

for i, comment in enumerate(comments, start=1):
    print(f"comment {i}: {comment}")

comment 1: I knew Jennifer Lawrence wasn&#39;t a real person
comment 2: Robot + feeling = love
comment 3: Beautiful
comment 4: Tu chizz badi hain mast 770 k dollers right
comment 5: Bluetooth
comment 6: DJs ❤
comment 7: she is the one that had remarks about the human race and if it would make it, she said ,no.
comment 8: <a href="https://www.youtube.com/watch?v=Bg_tJvCA8zw&amp;t=304">5:04</a> yep we’re doomed 💀
comment 9: 😮
comment 10: The interaction with Sophia is creepy and unsettling but I don’t think the robot noticed.
comment 11: Jimmy Fallon always looks, talks and gesters like a perv.
comment 12: Jimmy is so lovely 🤩
comment 13: Pourquoi n&#39;a-t-elle pas de cheveux ?
comment 14: Avengers: Age of Sophia
comment 15: “It’s my great start to dominate human race” jit ain’t jokin’
comment 16: Without a natural female human - society will never continue except via robot technology! That is truthful not fabricated deceit. God the true heavenly Good God our Heavenly Alnighty Lord God 

In [17]:
# Function to retrieve comments for a video
def get_video_comments(video_id):
    youtube = build('youtube', 'v3', developerKey=api_key)
    comments = []

    try:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            textFormat="plainText"
        )
        while request:
            response = request.execute()

            for item in response["items"]:
                comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
                comments.append(comment)

            request = youtube.commentThreads().list_next(request, response)
            
    except HttpError as e:
        if e.resp.status == 403:
            print(f"Comments are disabled for the video with ID {video_id}. Skipping...")
        else:
            raise
    return comments

# Function to check if comment dates are within 3 months of video publishing date
def within_3_months(video_published_date, comment_date):
    published_date = datetime.fromisoformat(video_published_date.replace("Z", ""))
    comment_date = datetime.fromisoformat(comment_date.replace("Z", ""))
    return comment_date <= published_date + timedelta(days=3*30)

# Function to collect comments for videos and filter by date
def collect_comments_with_date_check(videos_df):
    videos_df["comments"] = videos_df["video_id"].apply(get_video_comments)
    videos_df["comments_within_3_months"] = videos_df.apply(lambda row: [comment for comment in row["comments"] if within_3_months(row["video_publisheddate"], comment["publishedAt"])], axis=1)
    return videos_df

In [18]:
# Example usage: Search for videos on the topic of AI uploaded between 2017 and 2018
start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 1, 7)
videos_df = search_videos("AI", published_after=start_date, published_before=end_date)
videos_df = collect_comments_with_date_check(videos_df)

print(videos_df)

NameError: name 'HttpError' is not defined

In [13]:
# Example usage: Search for videos on the topic of AI uploaded between 2017 and 2023
start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 12, 31)
videos_df = search_videos("AI", published_after=start_date, published_before=end_date)


print(videos_df)

                                               title          publish_date  \
0  But what is a neural network? | Chapter 1, Dee...  2017-10-05T15:11:25Z   
1                       How AIs, like ChatGPT, Learn  2017-12-18T14:39:33Z   
2  Do Robots Deserve Rights? What if Machines Bec...  2017-02-23T13:31:05Z   
3    Google's DeepMind AI Just Taught Itself To Walk  2017-07-12T17:01:18Z   
4           AI - キラキラ feat.カンナ （KIRAKIRA feat.Kanna）  2017-09-01T03:56:38Z   
5                    Tay A.I. | The People's Chatbot  2017-09-10T14:10:17Z   
6                                What Makes Good AI?  2017-05-31T20:02:35Z   
7  The incredible inventions of intuitive AI | Ma...  2017-02-28T22:46:41Z   
8                       AI for Good - Sustainability  2017-05-18T17:51:42Z   
9                David, A.I  - Prometheus & Covenant  2017-10-23T15:00:20Z   

                                         description     video_id  view_count  
0  What are the neurons, why are there layers, an...  aircAru